In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
train = np.loadtxt("handout/train_triplets.txt", dtype=str, delimiter=" ")
test = np.loadtxt("handout/test_triplets.txt", dtype=str, delimiter=" ")

## Data manipulation

In [ ]:
img=mpimg.imread('handout/food/00000.jpg')
print(len(np.ndarray.flatten(img)))
plt.imshow(img);

In [ ]:
img_norm = tf.keras.utils.normalize(img, axis=1)
plt.imshow(img_norm);

In [ ]:
rows = []
img_size = 224
#for row in train[:2000]:
for row in train:
    #array = [mpimg.imread(f"handout/food/{n}.jpg") for n in row]
    #array = [img_to_array(load_img(f"handout/food/{n}.jpg")) for n in row]
    #array = [tf.keras.utils.normalize(cv2.imread(f"handout/food/{n}.jpg")) for n in row]
    #array = [cv2.imread(f"handout/food/{n}.jpg") for n in row]
    array = np.hstack([np.ndarray.flatten(cv2.resize(cv2.imread(f"handout/food/{n}.jpg"), (img_size, img_size))) for n in row])
    rows.append(array)
train_man = np.vstack(rows)

In [ ]:
if len(train)%2 != 0:
    idx = int((len(train_man) + 1) / 2)
else:
    idx = int(len(train_man) / 2)

train_man_one = train_man[:idx]
train_man_two = train_man[idx:]

sep = img_size * img_size * 3
t = np.copy(train_man_two[sep:2*sep])
train_man_two[sep:2*sep] = train_man_two[2*sep:]
train_man_two[2*sep:] = t

y_labels_one = np.ones(len(train_man_one))
y_labels_two = np.zeros(len(train_man_two))
y_labels = np.hstack([y_labels_one, y_labels_two])
x_train = np.vstack([train_man_one, train_man_two])

X_train, X_test, y_train, y_test = train_test_split(x_train, y_labels, test_size=0.2, random_state=42)

## Train

In [ ]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [ ]:
model.save("models/model_{}".format(datetime.today().strftime('%Y%m%d_%H%M%S')))

In [ ]:
# utility functions

def plot_eval(history):
    fig, ax = plt.subplots(1, 2, figsize=(24, 8))
    ax[0].plot(history.epoch, history.history["loss"], label="training loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="validation loss")
    ax[0].legend()
    ax[1].plot(history.epoch, history.history["accuracy"], label="training accuracy")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="validation accuracy")
    ax[1].legend()
    return fig, ax

In [ ]:
fig, ax = plot_eval(history);